# Notebook Data Preparation

This notebook shows how to extract data from wave files using extractor, and how the data is divided between test and training sets

## Chunk Data Prep

In [2]:
from models import gmm, lstm
from extractor import extractor
import os
import importlib
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
from collections import OrderedDict
import pprint as p

In [3]:
importlib.reload(extractor)
wav_data = "/tmp/data/voxforge" #Path to .wav file data
languages = {"english" : 0, "spanish" : 1, "french" : 2, "italian" : 3, "german" : 4}
TOTAL_FRAMES = 150
MIN_ENERGY = 12

extractor.TOTAL_FRAMES = TOTAL_FRAMES # lenght of audio each clip will be divided into
extractor.MIN_ENERGY = MIN_ENERGY # Minimum threshold of energy, zero's out anything not within threshold

In [4]:
data_y = []
data_x = []
# num_samples = 100
# get the path for num_samples .wav files
for language in languages:
    
    files = []
    path = os.path.join(wav_data, language)
    for f in os.listdir(path):
        files.append(os.path.join(path, f))
    
    x, y = extractor.make_feature_set(files, languages[language])
    data_x.append(x)
    data_y.append(y)

/Users/oosv/miniconda3/lib/python3.6/site-packages/mkl_fft/_numpy_fft.py:331: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  output = mkl_fft.rfft_numpy(a, n=n, axis=axis)


KeyboardInterrupt: 

In [5]:
print(len(data_y))
print(len(data_x))
data_x = np.concatenate(data_x, axis = 0)
data_y = np.concatenate(data_y, axis = 0)

5
5


In [7]:
shape_x, shape_y = np.shape(data_x), np.shape(data_y)
print(shape_x, shape_y)

(403374, 3, 150, 13) (403374,)


In [8]:
# Convert from Sup_Seq x 3 x TOTAL_FRAMES x n_coeff
#         to   Sup_Seq x TOTAL_FRAMES x n_coeff * 3
data_x = np.reshape(data_x, [shape_x[0], shape_x[2], shape_x[1] * shape_x[3]])
shape_x = np.shape(data_x)
print(shape_x)
print(f'total length of audio = {shape_x[0] * 1.50 / 3600} hours' )

(403374, 150, 39)
total length of audio = 168.0725 hours


In [9]:
def shuffle_in_unison(a, b):
    rng_state = np.random.get_state()
    np.random.shuffle(a)
    np.random.set_state(rng_state)
    np.random.shuffle(b)

In [11]:
counts = Counter(data_y)
print(counts)
total = sum(counts.values())
weights = {k : counts[k] / total for k in counts}
print(weights)

Counter({0: 154673, 1: 96687, 2: 63449, 4: 52284, 3: 36281})
{0: 0.38344811514872057, 1: 0.23969566704844636, 2: 0.15729571068041073, 3: 0.08994382384586017, 4: 0.1296166832765622}


In [11]:
# def make_equal(counts, data_x, data_y):
#     low_idx = min(counts, key=counts.get)
#     low = counts[low_idx]
#     data_x = data_x.tolist()
#     data_y = data_y.tolist()
#     #print(low)
#     new_y, new_x = [],[]
#     begin = 0
#     for idx in range(len(counts)):
#         new_y += data_y[begin: begin + low]
#         new_x += data_x[begin: begin + low]
#         #print(begin)
#         #print(begin + low)
#         begin += counts[idx]
#     return new_x, new_y


    

In [ ]:
counts = Counter(data_y_)
print(counts)

In [12]:
shuffle_in_unison(data_x, data_y)
shape_x = np.shape(data_x)

In [13]:
train_x = data_x[int(shape_x[0]/4):]
test_x = data_x[:int(shape_x[0]/4)]
train_y = data_y[int(shape_x[0]/4):]
test_y = data_y[:int(shape_x[0]/4)]

In [14]:
print(np.shape(train_x), np.shape(train_y))
print(np.shape(test_x), np.shape(test_y))

(302531, 150, 39) (302531,)
(100843, 150, 39) (100843,)


In [15]:
print(f'total_5_lang_audio =  {shape_x[0] * 1.50 / 3600} hours')

total_5_lang_audio =  168.0725 hours


In [16]:
with open('train_x_5.npy', 'wb') as file:
    np.save(file, train_x)
with open('train_y_5.npy', 'wb') as file:
    np.save(file, train_y)
with open('test_x_5.npy', 'wb') as file:
    np.save(file, test_x)
with open('test_y_5.npy', 'wb') as file:
    np.save(file, test_y)

## Full Utterance Data Prep

In [3]:
from models import gmm, lstm
from extractor import extractor
import os
import importlib
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
from collections import OrderedDict
import pprint as p

In [4]:
importlib.reload(extractor)
wav_data = "/Users/oosv/data/voxforge" #Path to .wav file data
languages = {"english" : 0, "spanish" : 1, "french" : 2, "italian" : 3, "german" : 4}
MIN_ENERGY = 12

extractor.MIN_ENERGY = MIN_ENERGY 

In [3]:
data_y = []
data_x = []
# num_samples = 100
# get the path for num_samples .wav files
for language in languages:
    
    files = []
    path = os.path.join(wav_data, language)
    for f in os.listdir(path):
        files.append(os.path.join(path, f))
    
    x, y = extractor.make_feature_set(files, languages[language], chunk=False)
    data_x.append(x)
    data_y.append(y)

/Users/oosv/miniconda3/lib/python3.6/site-packages/mkl_fft/_numpy_fft.py:331: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  output = mkl_fft.rfft_numpy(a, n=n, axis=axis)


In [7]:
print(len(data_x), len(data_y))

5 5


In [ ]:
# 0 pad all sequences to ensure equal dimensions for training purposes
# no special need for padding in case of test data though
def get_length_metrics(X):
    longest = 0
    sum_lengths = 0
    for idx, x in enumerate(X):
        cur = len(x[0])
        sum_lengths += cur
        if cur > longest:
            longest = cur
    avg = sum_lengths/(idx + 1)
    sum_diff = 0
    for idx, x in enumerate(X):
        sum_diff += (len(x[0]) - avg)**2
    std_dev = (sum_diff / idx) ** (1/2)
    return longest, avg, std_dev

In [9]:
def adjust_samples(X, avg, std_dev): 
    out = []
    for idx, x in enumerate(X):
        if len(x[0]) > avg + std_dev:
            shorted = np.array([x[0][:int(avg + std_dev)],
                       x[1][:int(avg + std_dev)],
                       x[2][:int(avg + std_dev)]])
            out.append(shorted)

        else:
            shape = np.shape(x)
            zeros = np.zeros([shape[0], int(avg + std_dev) - shape[1], shape[2]])
            out.append(np.concatenate((zeros, x), axis = 1))
    return np.array(out)

In [ ]:
def get_language_metrics(data_x):
    longest = 0
    metrics = {}
    for idx, lang in enumerate(data_x):
        metrics[idx] = get_length_metrics(lang)
    return metrics

In [ ]:
mets = get_language_metrics(data_x)
p.pprint (mets)

In [21]:
#lang_1 = adjust_samples(data_x[0],550, 250)
lang = adjust_samples(data_x[4], 550, 250)
    
with open('temp_{}.npy'.format(4), 'wb') as npy:
    np.save(npy, lang)

In [5]:
data_en = np.load('temp_0.npy')
np.shape(data_en)

(68365, 3, 800, 13)

In [6]:
data_es = np.load('temp_1.npy')
np.shape(data_es)

(23227, 3, 800, 13)

In [8]:
data_fr = np.load('temp_2.npy')
np.shape(data_fr)

(21657, 3, 800, 13)

In [10]:
data_it = np.load('temp_3.npy')
np.shape(data_it)

(10294, 3, 800, 13)

In [12]:
data_de = np.load('temp_4.npy')
np.shape(data_de)

(23937, 3, 800, 13)

# Data Preparation from Sagemaker Notebook

In [17]:
# Define IAM role
import boto3
import re
import os
import numpy as np
import pandas as pd
import importlib
from sagemaker import get_execution_role
import sagemaker as sage
from time import gmtime, strftime
from sagemaker.pytorch import PyTorch
import time

role         = get_execution_role()
sess         = sage.Session()
bucket       = 'oosv-multilingual-bucket' 
path         = '/tmp/data/npy/'
def get_data(file_name, bucket):
    file = os.path.join('/tmp/data/voxforge', file_name)
    s3 = boto3.resource('s3')
    s3.Bucket(bucket).download_file('data/tmp/' + file_name, file)
    data = np.load(file)
    os.remove(file)
    return data

path = '/home/ec2-user/SageMaker/data/npy/'
def upload_data(file_name, channel):
    target = f'data/{channel}/{channel}_5/'
    s3 = boto3.resource('s3')
    s3.Bucket(bucket).upload_file(os.path.join(path, file_name), target + file_name)
    
file_info = {"english" : "temp_0.npy",
             "spanish" : "temp_1.npy",
             "french"  : "temp_2.npy",
             "italitan": "temp_3.npy", 
             "german"  : "temp_4.npy"}

In [7]:
data_y = [0] * 68365 + [1] * 23227 + [2] * 21657 + [3] * 10294 + [4] * 23937

In [4]:
data_x = []
for lang in file_info:
    data_x.append(get_data(file_info[lang], bucket))

In [6]:
#data_x = np.concatenate(data_x, axis = 0)
print(np.shape(data_x))

(147480, 3, 800, 13)


In [8]:
def shuffle_in_unison(a, b):
    rng_state = np.random.get_state()
    np.random.shuffle(a)
    np.random.set_state(rng_state)
    np.random.shuffle(b)

In [9]:
shuffle_in_unison(data_x, data_y)
shape_x = np.shape(data_x)

In [10]:
train_x = data_x[int(shape_x[0]/4):]
test_x = data_x[:int(shape_x[0]/4)]
train_y = data_y[int(shape_x[0]/4):]
test_y = data_y[:int(shape_x[0]/4)]

In [11]:
print(np.shape(train_x), np.shape(train_y))
print(np.shape(test_x), np.shape(test_y))

(110610, 3, 800, 13) (110610,)
(36870, 3, 800, 13) (36870,)


In [12]:
print(f'total_5_lang_audio =  {shape_x[0] * .50 / 3600} hours')

total_5_lang_audio =  61.45 hours


In [20]:
file_name = 'full_train_x.npy'
with open(os.path.join(path,file_name), 'wb') as file:
    np.save(file, train_x)
upload_data(file_name, 'train')
os.remove(os.path.join(path,file_name))

file_name = 'full_train_y.npy'
with open(os.path.join(path,file_name), 'wb') as file:
    np.save(file, train_y)
upload_data(file_name, 'train')
os.remove(os.path.join(path,file_name))

file_name = 'full_test_x.npy'
with open(os.path.join(path,file_name), 'wb') as file:
    np.save(file, test_x)
upload_data(file_name, 'test')
os.remove(os.path.join(path,file_name))

file_name = 'full_test_y.npy'
with open(os.path.join(path,file_name), 'wb') as file:
    np.save(file, test_y)
upload_data(file_name, 'test')
os.remove(os.path.join(path,file_name))
